# EDA - Embeddings

In this notebook I will conduct EDA of node embeddings generated with help of two LLMs and one non-LLM method:
* OpenAI - generic text-embedding-3-small model with concurrency 50, currently implemented in our pipeline
* PubMedBert - biomedical embedding model, used in KGML-xDTD publication
* Spacy - generic pipeline with a pretrained language model - *en_core_web_md, web data training*
* SciSpacy - biomedical pipeline with a pretrained language model - *en_core_sci_md, biomedical data training*

**Summary:**

I created a (stratified) sample dataframe which consists of 93449 nodes (original has ~3.5 million nodes). For each node, I then generated embeddings using the models mentioned above (calculated them with a standalone script). Then I conducted a quantiative and qualitative analysis - compared distribution of embeddings, their similarity cosine/euclidean distance, PCA, tSNE. Finally I examined potential data leakage by comparing similarity of embeddings of drug nodes which are known to have treat/no treat relationship with disease nodes (based on GT dataset) - for better 'big picture' I also compared the similarity between drug nodes and disease nodes with no known relationships. 

Aditionally I examined mean different embedding generation approaches to compare Chunyus approach with different pooliong strategy - still WIP


ToC:
* Node Data - where I load node df from kedro, conduct stratified sampling based on category, and save subsamples locally
* Load embeddings - where I load embeddings (post-calculation which is done in a standalone script)
* Qualitative and quantitative assessment - comparison of embeddings through visualisation (tSNE, PCA), cosine similarity (post- and pre-PCA), distribution ks test
* Data Leakage Assessment - loading gt datasets and checking similarity between drug-disease pairs
* PubMedBERT vs PubMedBERT embeddings - compare LLM used by Chunyu vs sentence transformer of the same model

In [ ]:
import os
import time
import joblib
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
from scipy.spatial import distance
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

import pyspark as ps
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, concat_ws
import pyspark.sql.functions as f

# Setting the root path and changing the directory
root_path = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode().strip()
os.chdir(Path(root_path) / 'pipelines' / 'matrix')

%load_ext kedro.ipython
%reload_kedro  --env base

## Node data

In [ ]:
#load data 
input_nodes_raw = catalog.load('integration.model_input.nodes')
input_nodes_raw.show()

#show categories
input_nodes_raw.groupby('category').count().show()


In [ ]:
# for quick inspeciton we want to have a representative sample: we dont care about connectivity FOR NOW so we can create a 90 000 node
# this way we can also just obtain embeddings in a notebook (we can explore full embeddings later)

def stratified_sample(df, sample_size: int=90000, stratify_col: str='category'):
    total_count = df.count()
    category_counts = df.groupBy(stratify_col).count().collect()
    fractions = {row[stratify_col]: sample_size * (row['count'] / total_count) / row['count'] for row in category_counts}
    
    print(fractions)
    sampled_df = df.sampleBy(stratify_col, fractions, seed=42)
    return sampled_df, fractions

sample, fracs = stratified_sample(input_nodes_raw)

sample.show()
sample.count()) 

In [ ]:
sample_df

The subsample has 93 000 nodes (contrary to original )

In [ ]:
# get features from the sample df and save
sample = sample_spark.withColumn("feat", concat_ws("+", col("name"), col("category")))
features = sample.select('feat').rdd.flatMap(lambda x:x).collect()
joblib.dump(features, 'sm_sample_features.joblib')

#save sample df in parquet
spark = SparkSession \
    .builder \
    .appName("Protob Conversion to Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sample.write.parquet("sm_sample_df.parquet")


## Load Embeddings

I calculated these embeddings using a python script (compute_embeddings) - code is below. PubMedBERT was much slower than OpenAI for the same subset and using the same batches (not entirely comparables as OpenAI has an API call but this still shows spacy being much faster)

PubMedBERT for subsample 5164 s
OpenAI for subsample 3120 s
spaCy for subsample 1200s (with ner included )
sciSpacy for subsamle - 920s (ner excl)

In [ ]:
#load embedding: turn into df

#pubmedbert
pubmed_emb = joblib.load('scratch/pubmedbert_sm_embed.joblib')

#openai
openai_emb = np.array(joblib.load('scratch/openai_sm_embed.joblib'))

#spacy
spacy_emb = joblib.load('scratch/spacy_md_sm_embed.joblib')

#scispacy
scispacy_emb = joblib.load('scratch/scispacy_sm_embed.joblib')

## Qualitative and Quantitative assessment

In [ ]:
#check distribution of embeddings
plt.hist(np.array(pubmed_emb).flatten(), bins=50, alpha=0.5, label='pubmed')
plt.hist(np.array(openai_emb).flatten(), bins=50, alpha=0.5, label='openai')
plt.hist(np.array(spacy_emb).flatten(), bins=50, alpha=0.5, label='spacy')
plt.hist(np.array(scispacy_emb).flatten(), bins=50, alpha=0.5, label='scispacy')
plt.legend(loc='upper right')
plt.suptitle('distribution of values in embeddings')
plt.savefig('distribution_plot.png')
plt.show()


In [ ]:
#quantitative assessment - distance between PubMedBERT and OpenAI only as they have the same length

dist_dict = {'pubmed':np.array(pubmed_emb),'openai':np.array(openai_emb)}

for name in dist_dict.keys():
    print(name)
    new_dict = dist_dict.copy()
    new_dict.pop(name)
    for name2 in new_dict.keys():
        euc_dist=[]
        cos_dist=[]
        for i in range(len(dist_dict[name])):
            euc_dist.append(distance.euclidean(dist_dict[name][i], new_dict[name2][i]))
            cos_dist.append(cosine_similarity(dist_dict[name][i].reshape(1,-1), new_dict[name2][i].reshape(1,-1)))

print('euc')
print('mean', np.mean(euc_dist), 'std', np.std(euc_dist))
print('cos')
print('mean', np.mean(cos_dist), 'std', np.std(cos_dist))

#sanity to check they come from diff distributions
print(ks_2samp(np.array(pubmed_emb).flatten(), np.array(openai_emb).flatten()))

In [ ]:
# PCA to ensure they are all 100 components long
from sklearn.decomposition import PCA
import pandas as pd

pca = PCA(n_components=100)
pubmed_pca = pca.fit_transform(pubmed_emb)
openai_pca = pca.fit_transform(openai_emb)
spacy_pca = pca.fit_transform(spacy_emb)
scispacy_pca = pca.fit_transform(scispacy_emb)

In [ ]:
# compare similarity post pca

dist_dict = {'pubmed':pubmed_pca,'openai':openai_pca, 'scispacy':scispacy_pca, 'spacy':spacy_pca}

for name in dist_dict.keys():
    new_dict = dist_dict.copy()
    new_dict.pop(name)
    for name2 in new_dict.keys():
        print(name, '-', name2)
        euc_dist=[]
        cos_dist=[]
        for i in range(len(dist_dict[name])):
            euc_dist.append(distance.euclidean(dist_dict[name][i], new_dict[name2][i]))
            cos_dist.append(cosine_similarity(dist_dict[name][i].reshape(1,-1), new_dict[name2][i].reshape(1,-1)))
        print('euc distance results')
        print('mean', np.mean(euc_dist), 'std', np.std(euc_dist))
        print('cosine similarity results')
        print('mean', np.mean(cos_dist), 'std', np.std(cos_dist), '\n')


Based on the distribution, distance & similarity checks so far:
* no embeddings are identical or close to identical
* the histogram shows that the distribution partially overlaps for OpenAI, PubMedBERT and SciSpacy but definitely not the same distribution
* spacy embeddings are 'an outcast' as they are very different from remaining embeddings
* openai and pubmedbert are the most similar to each other out of four, however scispacy is not hugely different either

### tSNE - 2 components

tSNE should be able to indicate the ability of the models to cluster nodes and how well they align with their categories.

In [ ]:
tsne = TSNE(n_components=2, perplexity=30.0)
pubmed_tsne = tsne.fit_transform(np.array(pubmed_emb))
openai_tsne = tsne.fit_transform(np.array(openai_emb))
spacy_tsne = tsne.fit_transform(np.array(spacy_emb))
scispacy_tsne = tsne.fit_transform(np.array(scispacy_emb))

#turn into df, concat with category for tsne
pubmed_emb_df = pd.DataFrame(pubmed_tsne)
openai_emb_df = pd.DataFrame(openai_tsne)
spacy_emb_df = pd.DataFrame(spacy_tsne)
scispacy_emb_df = pd.DataFrame(scispacy_tsne)

#read sample metadata
spark = SparkSession \
    .builder \
    .appName("Protob Conversion to Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

metadata=spark.read.parquet('sm_sample_df.parquet').toPandas()

#concat
pubmed_emb_df = pd.concat([metadata,pubmed_emb_df], axis=1)
openai_emb_df = pd.concat([metadata,openai_emb_df], axis=1)
spacy_emb_df = pd.concat([metadata,spacy_emb_df], axis=1)
scispacy_emb_df = pd.concat([metadata,scispacy_emb_df], axis=1)

In [ ]:
metadata.groupby('category').count()

In [ ]:

#modify colors per category
plt.subplots(1,4, figsize=(25,10))
plt.subplot(1,4,1)
sns.scatterplot(data = pubmed_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('pubmed')
plt.xlabel('tsne 1')
plt.ylabel('tsne 2')
plt.subplot(1,4,2)
sns.scatterplot(data = spacy_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('spacy')
plt.xlabel('tsne 1')
plt.ylabel('tsne 2')
plt.subplot(1,4,3)
sns.scatterplot(data = openai_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('openai')
plt.xlabel('tsne 1')
plt.ylabel('tsne 2')
plt.subplot(1,4,4)
sns.scatterplot(data = scispacy_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('scispacy')
plt.xlabel('tsne 1')
plt.ylabel('tsne 2')
plt.show()
plt.tight_layout()

PubMedBERT seems to have the nicestt clusters and least scattered points but openai embeddings are relatively similar (bit more scattered). Spacy fails at capturing categories together, scispacy does it better but not as good as LLMs

### PCA 2 components

In [ ]:


pca = PCA(n_components=2)
pubmed_pca = pca.fit_transform(pubmed_emb)
openai_pca = pca.fit_transform(openai_emb)
#spacy_md_pca = pca.fit_transform(spacy_md_emb)
spacy_pca = pca.fit_transform(spacy_emb)
scispacy_pca = pca.fit_transform(scispacy_emb)

#turn into df, concat with category for pca
pubmed_emb_df = pd.DataFrame(pubmed_pca)
openai_emb_df = pd.DataFrame(openai_pca)
spacy_emb_df = pd.DataFrame(spacy_pca)
scispacy_emb_df = pd.DataFrame(scispacy_pca)

metadata=spark.read.parquet('sm_sample_df.parquet').toPandas()

#concat
pubmed_emb_df = pd.concat([metadata,pubmed_emb_df], axis=1)
openai_emb_df = pd.concat([metadata,openai_emb_df], axis=1)
spacy_emb_df = pd.concat([metadata,spacy_emb_df], axis=1)
scispacy_emb_df = pd.concat([metadata,scispacy_emb_df], axis=1)

In [ ]:
import seaborn as sns

#modify colors per category
plt.subplots(1,4, figsize=(25,10))
plt.subplot(1,4,1)
sns.scatterplot(data = pubmed_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('pubmed')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.subplot(1,4,2)
sns.scatterplot(data = spacy_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('spacy')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.subplot(1,4,3)
sns.scatterplot(data = openai_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('openai')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.subplot(1,4,4)
sns.scatterplot(data = scispacy_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('scispacy')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()
plt.tight_layout()

In [ ]:

#modify colors per category
plt.subplots(1,4, figsize=(25,10), sharex=True, sharey=True)
plt.subplot(1,4,1)
sns.scatterplot(data = pubmed_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('pubmed')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.subplot(1,4,2)
sns.scatterplot(data = spacy_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('spacy')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.subplot(1,4,3)
sns.scatterplot(data = openai_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('openai')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.subplot(1,4,4)
sns.scatterplot(data = scispacy_emb_df, x=0, y=1, hue='category', alpha=0.2, legend=False)
plt.title('scispacy')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()
plt.tight_layout()

PubMedBERT seems to have much greater variance within the embeddings compared to remaining models, especially OpenAI. This is consistent with the distribution plots shown earlieir. Could indicate that PubMedBERT embeddings are more 'diverse' and informative than the remaining ones but might also mean that data leakage is much more likely the case

## Examine similarity between GT for data leakage detection

In [ ]:
import sys
import pandas as pd

#read gt
#gt = catalog.load('integration.int.known_pairs@pandas')
#gt.to_csv('scratch/gt.csv')
gt = pd.read_csv('scratch/gt.csv').drop('Unnamed: 0',axis=1)
gt

In [ ]:
# spark 
spark = SparkSession \
    .builder \
    .appName("Protob Conversion to Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sample_df = spark.read.parquet("sm_sample_df.parquet").toPandas()
sample_df

#create a subsample which contains both diseases and drugs from gt
sampled_df_gt = sample_df.loc[( sample_df.id.isin(gt.source) | sample_df.id.isin(gt.target) )]


### PubmedBERT

In [ ]:
drugs_gt_df=sampled_df_gt[sampled_df_gt.category.isin(['biolink:Drug','biolink:SmallMolecule'])]

treat_pubmed_cos_sim_list=[]
no_treat_pubmed_cos_sim_list=[]
unknown_treat_pubmed_cos_sim_list=[]
pubmed_cos_sim_list=[]
pubmed_dot_prod=[]
pubmed_euc=[]
for node_id in drugs_gt_df.index:
    drug_name = drugs_gt_df.loc[node_id, 'id']
    drug_node = pubmed_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        disease_node = pubmed_emb[target_id]
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            relation='treats'
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            relation='not treats'
        else:
            relation='unknown if treats'
        print(drug_name, relation, target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1)))
        pubmed_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        dot_product=np.dot(drug_node,disease_node)
        print('Dot product: ',dot_product)
        pubmed_dot_prod.append(dot_product)
        euc_dist=distance.euclidean(drug_node,disease_node)
        print('Euclidean distance: ',euc_dist)
        pubmed_euc.append(euc_dist)
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            treat_pubmed_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            no_treat_pubmed_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        else:
            unknown_treat_pubmed_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])

In [ ]:
plt.subplot(1,3,1)
sns.kdeplot(pubmed_cos_sim_list)
plt.subplot(1,3,2)
sns.kdeplot(pubmed_dot_prod)
plt.subplot(1,3,3)
sns.kdeplot(pubmed_euc)
plt.tight_layout()
print('Mean ', np.mean(pubmed_cos_sim_list))
print('std ',np.std(pubmed_cos_sim_list))
print('Min ',min(pubmed_cos_sim_list))
print('Max ',max(pubmed_cos_sim_list))

In [ ]:
plt.subplot(1,3,1)
sns.kdeplot(pubmed_cos_sim_list)
plt.title('all gt pairs')
plt.subplot(1,3,2)
sns.kdeplot(treat_pubmed_cos_sim_list, label='y=1', legend=True)
#plt.subplot(1,3,3)
sns.kdeplot(no_treat_pubmed_cos_sim_list, label='y=0', legend=True)
plt.legend()

plt.tight_layout()

#### PubmedBERT - random pairs

In [ ]:
# check for non drug-disase pairs
pubmed_cos_sim_list_nodd=[]
for i, node_id in enumerate(drugs_gt_df.index):
    drug_name = drugs_gt_df.loc[node_id, 'id']
    if i!=88:
        num = i+1
    else:
        num = 0
    drug_name_next = drugs_gt_df.loc[drugs_gt_df.index[num], 'id']
    drug_node = pubmed_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name_next)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        if gt.loc[(gt.source==drug_name) & (gt.target==target)].shape!=(0,3):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        disease_node = pubmed_emb[target_id]
        print(drug_name, '-', target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1)))
        pubmed_cos_sim_list_nodd.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
    

In [ ]:
sns.kdeplot(pubmed_cos_sim_list_nodd)
print('Mean ', np.mean(pubmed_cos_sim_list_nodd))
print('std ',np.std(pubmed_cos_sim_list_nodd))
print('Min ',min(pubmed_cos_sim_list_nodd))
print('Max ',max(pubmed_cos_sim_list_nodd))


### OpenAI

In [ ]:
drugs_gt_df=sampled_df_gt[sampled_df_gt.category.isin(['biolink:Drug','biolink:SmallMolecule'])]

treat_openai_cos_sim_list=[]
no_treat_openai_cos_sim_list=[]
unknown_treat_openai_cos_sim_list=[]
openai_cos_sim_list=[]
openai_dot_prod=[]
openai_euc=[]
for node_id in drugs_gt_df.index:
    drug_name = drugs_gt_df.loc[node_id, 'id']
    drug_node = openai_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        disease_node = openai_emb[target_id]
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            relation='treats'
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            relation='not treats'
        else:
            relation='unknown if treats'
        print(drug_name, relation, target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1)))
        openai_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        dot_product=np.dot(drug_node,disease_node)
        print('Dot product: ',dot_product)
        openai_dot_prod.append(dot_product)
        euc_dist=distance.euclidean(drug_node,disease_node)
        print('Euclidean distance: ',euc_dist)
        openai_euc.append(euc_dist)
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            treat_openai_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            no_treat_openai_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        else:
            unknown_treat_openai_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])


In [ ]:
sns.kdeplot(openai_cos_sim_list)
print('Mean ', np.mean(openai_cos_sim_list))
print('std ',np.std(openai_cos_sim_list))
print('Min ',min(openai_cos_sim_list))
print('Max ',max(openai_cos_sim_list))

In [ ]:

plt.subplot(1,3,1)
sns.kdeplot(openai_cos_sim_list)
plt.title('all gt pairs')
plt.subplot(1,3,2)
sns.kdeplot(treat_openai_cos_sim_list, label='y=1', legend=True)
sns.kdeplot(no_treat_openai_cos_sim_list, label='y=0', legend=True)
plt.legend()

plt.tight_layout()

#### OpenAI - random pairs

In [ ]:
# check for non drug-disase pairs
openai_cos_sim_list_nodd=[]
for i, node_id in enumerate(drugs_gt_df.index):
    drug_name = drugs_gt_df.loc[node_id, 'id']
    if i!=88:
        num = i+1
    else:
        num = 0
    drug_name_next = drugs_gt_df.loc[drugs_gt_df.index[num], 'id']
    drug_node = openai_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name_next)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        print(gt.loc[(gt.source==drug_name) & (gt.target==target)].shape)
        if gt.loc[(gt.source==drug_name) & (gt.target==target)].shape!=(0,3):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        disease_node = openai_emb[target_id]
        print(drug_name_next, '-', target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1)))
        openai_cos_sim_list_nodd.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
    

In [ ]:
sns.kdeplot(openai_cos_sim_list_nodd)
print('Mean ', np.mean(openai_cos_sim_list_nodd))
print('std ',np.std(openai_cos_sim_list_nodd))
print('Min ',min(openai_cos_sim_list_nodd))
print('Max ',max(openai_cos_sim_list_nodd))

### Spacy

In [ ]:
drugs_gt_df=sampled_df_gt[sampled_df_gt.category.isin(['biolink:Drug','biolink:SmallMolecule'])]

treat_spacy_cos_sim_list=[]
no_treat_spacy_cos_sim_list=[]
unknown_treat_spacy_cos_sim_list=[]
spacy_cos_sim_list=[]
spacy_dot_prod=[]
spacy_euc=[]
for node_id in drugs_gt_df.index:
    drug_name = drugs_gt_df.loc[node_id, 'id']
    drug_node = spacy_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        disease_node = spacy_emb[target_id]
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            relation='treats'
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            relation='not treats'
        else:
            relation='unknown if treats'
        print(drug_name, relation, target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1)))
        spacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        dot_product=np.dot(drug_node,disease_node)
        print('Dot product: ',dot_product)
        spacy_dot_prod.append(dot_product)
        euc_dist=distance.euclidean(drug_node,disease_node)
        print('Euclidean distance: ',euc_dist)
        spacy_euc.append(euc_dist)
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            treat_spacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            no_treat_spacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        else:
            unknown_treat_spacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])


In [ ]:
sns.kdeplot(spacy_cos_sim_list)

print('Mean ', np.mean(spacy_cos_sim_list))
print('std ',np.std(spacy_cos_sim_list))
print('Min ',min(spacy_cos_sim_list))
print('Max ',max(spacy_cos_sim_list))

In [ ]:

plt.subplot(1,3,1)
sns.kdeplot(spacy_cos_sim_list)
plt.title('all gt pairs')
plt.subplot(1,3,2)
sns.kdeplot(treat_spacy_cos_sim_list, label='y=1', legend=True)
sns.kdeplot(no_treat_spacy_cos_sim_list, label='y=0', legend=True)
plt.legend()

plt.tight_layout()

#### Spacy - random

In [ ]:
# check for non drug-disase pairs
spacy_cos_sim_list_nodd=[]
for i, node_id in enumerate(drugs_gt_df.index):
    drug_name = drugs_gt_df.loc[node_id, 'id']
    if i!=88:
        num = i+1
    else:
        num = 0
    drug_name_next = drugs_gt_df.loc[drugs_gt_df.index[num], 'id']
    drug_node = spacy_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name_next)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        print(gt.loc[(gt.source==drug_name) & (gt.target==target)].shape)
        if gt.loc[(gt.source==drug_name) & (gt.target==target)].shape!=(0,3):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        other_node = spacy_emb[target_id]
        print(drug_name_next, '-', target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), other_node.reshape(1, -1)))
        spacy_cos_sim_list_nodd.append(cosine_similarity(drug_node.reshape(1, -1), other_node.reshape(1, -1))[0][0])
    

In [ ]:
sns.kdeplot(spacy_cos_sim_list_nodd)

print('Mean ', np.mean(spacy_cos_sim_list_nodd))
print('std ',np.std(spacy_cos_sim_list_nodd))
print('Min ',min(spacy_cos_sim_list_nodd))
print('Max ',max(spacy_cos_sim_list_nodd))

### SciSPacy

In [ ]:
drugs_gt_df=sampled_df_gt[sampled_df_gt.category.isin(['biolink:Drug','biolink:SmallMolecule'])]

treat_scispacy_cos_sim_list=[]
no_treat_scispacy_cos_sim_list=[]
unknown_treat_scispacy_cos_sim_list=[]
scispacy_cos_sim_list=[]
scispacy_dot_prod=[]
scispacy_euc=[]
for node_id in drugs_gt_df.index:
    drug_name = drugs_gt_df.loc[node_id, 'id']
    drug_node = scispacy_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        disease_node = scispacy_emb[target_id]
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            relation='treats'
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            relation='not treats'
        else:
            relation='unknown if treats'
        print(drug_name, relation, target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1)))
        scispacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        dot_product=np.dot(drug_node,disease_node)
        print('Dot product: ',dot_product)
        scispacy_dot_prod.append(dot_product)
        euc_dist=distance.euclidean(drug_node,disease_node)
        print('Euclidean distance: ',euc_dist)
        scispacy_euc.append(euc_dist)
        if gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==1:
            treat_scispacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        elif gt.loc[(gt.source==drug_name)&(gt.target==target)].y.values[0]==0:
            no_treat_scispacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])
        else:
            unknown_treat_scispacy_cos_sim_list.append(cosine_similarity(drug_node.reshape(1, -1), disease_node.reshape(1, -1))[0][0])


In [ ]:
sns.kdeplot(scispacy_cos_sim_list)
print('Mean ', np.mean(scispacy_cos_sim_list))
print('std ',np.std(scispacy_cos_sim_list))
print('Min ',min(scispacy_cos_sim_list))
print('Max ',max(scispacy_cos_sim_list))

In [ ]:
plt.subplot(1,3,1)
sns.kdeplot(scispacy_cos_sim_list)
plt.title('all gt pairs')
plt.subplot(1,3,2)
sns.kdeplot(treat_scispacy_cos_sim_list, label='y=1', legend=True)
sns.kdeplot(no_treat_scispacy_cos_sim_list, label='y=0', legend=True)
plt.legend()

plt.tight_layout()

#### Scispacy - random

In [ ]:
# check for non drug-disase pairs
scispacy_cos_sim_list_nodd=[]
for i, node_id in enumerate(drugs_gt_df.index):
    drug_name = drugs_gt_df.loc[node_id, 'id']
    if i!=88:
        num = i+1
    else:
        num = 0
    drug_name_next = drugs_gt_df.loc[drugs_gt_df.index[num], 'id']
    drug_node = scispacy_emb[node_id]
    targets = []
    targets.extend(gt.loc[(gt.source==drug_name_next)].target.values)
    for target in targets:
        if all(sampled_df_gt.id!=target):
            continue
        print(gt.loc[(gt.source==drug_name) & (gt.target==target)].shape)
        if gt.loc[(gt.source==drug_name) & (gt.target==target)].shape!=(0,3):
            continue
        target_id =int(sampled_df_gt.loc[target==sampled_df_gt.id].index.values)
        other_node = scispacy_emb[target_id]
        print(drug_name_next, '-', target, end='\t')
        print('Cosine similarity :', cosine_similarity(drug_node.reshape(1, -1), other_node.reshape(1, -1)))
        scispacy_cos_sim_list_nodd.append(cosine_similarity(drug_node.reshape(1, -1), other_node.reshape(1, -1))[0][0])
    

In [ ]:
sns.kdeplot(scispacy_cos_sim_list_nodd)
print('Mean ', np.mean(scispacy_cos_sim_list_nodd))
print('std ',np.std(scispacy_cos_sim_list_nodd))
print('Min ',min(scispacy_cos_sim_list_nodd))
print('Max ',max(scispacy_cos_sim_list_nodd))

### Summary

In [ ]:
plt.subplots(1,4, figsize=(15,5), sharex=True, sharey=True)
plt.subplot(1,4,1)
plt.title('Pubmed')
sns.kdeplot(pubmed_cos_sim_list, label='Pubmed')
plt.subplot(1,4,2)
plt.title('OpenAI')
sns.kdeplot(openai_cos_sim_list, label='OpenAI')
plt.subplot(1,4,3)
plt.title('Spacy')
sns.kdeplot(spacy_cos_sim_list, label='Spacy')
plt.subplot(1,4,4)
plt.title('SciSpacy')
sns.kdeplot(scispacy_cos_sim_list, label='SciSpacy')
plt.suptitle('Cosine similarity distribution for drug-disease pairs in GT')
plt.show()

In [ ]:
plt.subplots(1,4, figsize=(15,5), sharex=True, sharey=True)
plt.subplot(1,4,1)
plt.title('Pubmed')
sns.kdeplot(treat_pubmed_cos_sim_list, label='y=1', legend=True)
sns.kdeplot(no_treat_pubmed_cos_sim_list, label='y=0', legend=True)
plt.legend()
plt.subplot(1,4,2)
plt.title('OpenAI')
sns.kdeplot(treat_openai_cos_sim_list, label='y=1', legend=True)
sns.kdeplot(no_treat_openai_cos_sim_list, label='y=0', legend=True)
plt.legend()
plt.subplot(1,4,3)
plt.title('Spacy')
sns.kdeplot(treat_spacy_cos_sim_list, label='y=1', legend=True)
sns.kdeplot(no_treat_spacy_cos_sim_list, label='y=0', legend=True)
plt.legend()
plt.subplot(1,4,4)
plt.title('SciSpacy')
sns.kdeplot(treat_scispacy_cos_sim_list, label='y=1', legend=True)
sns.kdeplot(no_treat_scispacy_cos_sim_list, label='y=0', legend=True)
plt.suptitle('Cosine similarity distribution fordrug-disease pairs ')
plt.legend()
plt.show()

In [ ]:
plt.subplots(1,4, figsize=(15,5), sharex=True, sharey=True)
plt.subplot(1,4,1)
plt.title('Pubmed')
sns.kdeplot(pubmed_cos_sim_list, label='Pubmed')
sns.kdeplot(pubmed_cos_sim_list_nodd, label='random')
plt.legend()
plt.subplot(1,4,2)
plt.title('OpenAI')
sns.kdeplot(openai_cos_sim_list, label='random')
sns.kdeplot(openai_cos_sim_list_nodd, label='random')
plt.legend()
plt.subplot(1,4,3)
plt.title('Spacy')
sns.kdeplot(spacy_cos_sim_list, label='random')
sns.kdeplot(spacy_cos_sim_list_nodd, label='random')
plt.legend()
plt.subplot(1,4,4)
plt.title('SciSpacy')
sns.kdeplot(scispacy_cos_sim_list, label='random')
sns.kdeplot(scispacy_cos_sim_list_nodd, label='random')
plt.legend()
plt.suptitle('Cosine similarity distribution for random vs gt drug-disease pairs ')
plt.legend()
plt.show()

## PubMedBERT vs PubMedBERT embeddings - WIP

Chunyu generated pubmedbert embeddings with base pubmedbert, but there is a specific embedding model from pubmedbert on HF - could be much faster. The model is actually using mean pooling (which is common when extracting embeddings) but what Chunyus is using is simply pooler output

In [ ]:
def meanpooling(output, mask):
    embeddings = output[0] # First element of model_output contains all token embeddings
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)

t1=time.time()
sentences = features
tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
with torch.no_grad():
    output = model(**inputs, output_hidden_states=True, return_dict=True)
embeddings = meanpooling(output, inputs['attention_mask'])

print("Sentence embeddings:")
print(embeddings)
t2=time.time()
print(t2-t1)

In [ ]:
features = joblib.load('sm_sample_features.joblib')[:1000]
sentences = features
t1=time.time()
model = SentenceTransformer("neuml/pubmedbert-base-embeddings")
embeddings0 = model.encode(sentences)
print(embeddings)
t2=time.time()
print(t2-t1)

In [ ]:

t1=time.time()
tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
with torch.no_grad():
    embeddings1 = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
print(embeddings1)
t2=time.time()
print(t2-t1)

In [ ]:

t1=time.time()
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
with torch.no_grad():
    embeddings2 = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
print(embeddings2)
t2=time.time()
print(t2-t1)

In [ ]:

t1=time.time()
def meanpooling(output, mask):
    embeddings = output[0]
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)

with torch.no_grad():
    output = model(**inputs, output_hidden_states=True, return_dict=True)

embeddings = meanpooling(output, inputs['attention_mask'])

print("Sentence embeddings:")
print(embeddings)
t2=time.time()
print(t2-t1)

In [ ]:


#same?
print(ks_2samp(np.array(embeddings1).flatten(), np.array(embeddings2).flatten()))
print(ks_2samp(np.array(embeddings1).flatten(), np.array(embeddings).flatten()))
print(ks_2samp(np.array(embeddings).flatten(), np.array(embeddings2).flatten()))


In [ ]:
#compare embed1 and embed2
euc_dist = []
cos_dist = []
for i in range(len(embeddings1)):
    euc_dist.append(distance.euclidean(np.array(embeddings1[i]), np.array(embeddings2[i])))
    cos_dist.append(distance.cosine(np.array(embeddings1[i]), np.array(embeddings2[i])))
print('Euc mean ',np.mean(euc_dist), ' std ', np.std(euc_dist))
print('Cos mean ',np.mean(cos_dist), ' std ', np.std(cos_dist))

In [ ]:
#compare embed1 and embed
euc_dist = []
cos_dist = []
for i in range(len(embeddings1)):
    euc_dist.append(distance.euclidean(np.array(embeddings1[i]), np.array(embeddings[i])))
    cos_dist.append(distance.cosine(np.array(embeddings1[i]), np.array(embeddings[i])))
print('Euc mean ',np.mean(euc_dist), ' std ', np.std(euc_dist))
print('Cos mean ',np.mean(cos_dist), ' std ', np.std(cos_dist))

In [ ]:
#compare embed2 and embed
euc_dist = []
cos_dist = []
for i in range(len(embeddings1)):
    euc_dist.append(distance.euclidean(np.array(embeddings2[i]), np.array(embeddings[i])))
    cos_dist.append(distance.cosine(np.array(embeddings2[i]), np.array(embeddings[i])))
print('Euc mean ',np.mean(euc_dist), ' std ', np.std(euc_dist))
print('Cos mean ',np.mean(cos_dist), ' std ', np.std(cos_dist))

Sentence transformer is much faster than the tokenizer -> model -> hidden layers: the embeddings differ from when they are obtained through NeuML or original base model. Question to be answered tho: why did Chunyu use pooler_output rather than mean pooling?

## Next steps

* run graphsage on the subsets of embeddings and commpare how topological embeddings differ
* train ML classifier to predict drug-disease interactions to see how the performance differs